In [3]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("spotify_millsongdata.csv")
df.head()

In [10]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [11]:
df.head(10)

,artist,song,text
0,Alabama,Hangin' Round The Mistletoe,"Snowy days and frosty night, everything looks ..."
1,O.A.R.,Get Away,Is it me or does it seem \r\nThat life just c...
2,America,Garden Of Peace,I look inside at the scarlet room \r\nPlaces ...
3,Barbra Streisand,For Pete's Sake,People are born to love and be happy \r\nLivi...
4,Miley Cyrus,FU,Oh you broke my heart \r\nI told you I was we...
5,Dan Fogelberg,Stolen Moments,"Every time we speak, our words betray intentio..."
6,Phish,Gumbo,There's a mouse starin' out of the window \r\...
7,Morrissey,At Last I Am Born,At last I am born \r\nHistorians note \r\nI ...
8,James Taylor,Another Day,"Wake up Suzy, put your shoes on, \r\nWalk wit..."
9,John Mellencamp,Where The World Began,I'm tired of being grateful for a chance to ta...


In [12]:
df['text'][0]

"Snowy days and frosty night, everything looks cold and white  \r\nHangin' 'round the mistletoe, hangin' 'round the mistletoe  \r\n  \r\nChristmas day will soon be here, and everybody's full of cheer  \r\nHangin' 'round the mistletoe, waitin' for you dear  \r\n  \r\nWell, I've waited all year long and nore the year is almost gone  \r\nHangin' 'round the mistletoe, hangin' 'round the mistletoe  \r\n  \r\nBut if that's what I've gotta do just to get a kiss from you  \r\nHangin' 'round the mistletoe, waitin' for you dear  \r\n  \r\nChristmas is a special time to give and to receive  \r\nFor every little kiss you give me I will give you three  \r\n  \r\nYou ain't gonna miss one kiss, that's one thing you'll never miss  \r\nI'm hangin' 'round the mistletoe waitin' for you dear  \r\n  \r\nWell, I've had my eyes on you for a long, long time  \r\nSo, I'm asking Santa please, make that pretty girl mine  \r\n  \r\nWaited till I'm cold and blue just to get a kiss from you  \r\nI'm hangin' 'round 

Text cleaning

In [13]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n',' ',regex=True)

In [14]:
df.tail(5)

,artist,song,text
4995,Ufo,At War With The World,hit me like a hammer to my head \r we thrashe...
4996,ABBA,Ahe's My Kind Of Girl,"look at her face, it's a wonderful face \r an..."
4997,Steve Miller Band,Fanny Mae,"i know, my baby told me so \r yes, i know, my..."
4998,Rascal Flatts,Falling Upside Down,there's no gravity up here \r it's kind of ha...
4999,Steve Miller Band,Behind The Barn,"well, i went to see my woman \r way back down..."


In [15]:
import nltk
from nltk.stem.porter import PorterStemmer

In [16]:
stemmer= PorterStemmer()

In [17]:
def token(txt):
     token=nltk.word_tokenize(txt)
     a =[stemmer.stem(w) for w in token]
     return " ".join(a)

In [18]:
df['text'].apply(lambda x: token(x))

0       snowi day and frosti night , everyth look cold...
1       is it me or doe it seem that life just carri o...
2       i look insid at the scarlet room place to hide...
3       peopl are born to love and be happi live a sun...
4       oh you broke my heart i told you i wa weak for...
                              ...                        
4995    hit me like a hammer to my head we thrash in b...
4996    look at her face , it 's a wonder face and it ...
4997    i know , my babi told me so ye , i know , my b...
4998    there 's no graviti up here it 's kind of hard...
4999    well , i went to see my woman way back down on...
Name: text, Length: 5000, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [21]:
matrix=tfid.fit_transform(df['text'])

In [36]:
similerity=cosine_similarity(matrix)

In [37]:
similerity[0]

array([1.        , 0.01791995, 0.0019112 , ..., 0.00662423, 0.00318141,
       0.00408809])

In [38]:
df[df['song']=='Falling Upside Down']

,artist,song,text
4998,Rascal Flatts,Falling Upside Down,there's no gravity up here \r it's kind of ha...


Recommender Function

In [48]:
def recommender(song_df):
    idx=df[df['song']==song_df].index[0]
    distances=sorted(list(enumerate(similerity[idx])),reverse=True, key=lambda x:x[1])
    songs =[]
    for m_id in distances[1:20]:
        songs.append(df.iloc[m_id[0]].song)
    return songs

In [49]:
recommender('Falling Upside Down')

['Kite Song',
 'On The Wings Of Love',
 "I've Got The World On A String",
 'Free As A Bird',
 'I Guess I Like It Like That',
 'Nothing Like The First Time',
 'I Like To',
 'I Like You',
 'Eventually',
 'Light Lift Me Up',
 'The Upside',
 'Only Girl In The World',
 'Heart By Heart',
 "Fallin'",
 'Greatest Show On Earth',
 'Ancients',
 'Nobody Sees Me Like You Do',
 'Hymn For The Weekend',
 'Sweet Young Thing Like You']

In [43]:
import pickle

In [44]:
pickle.dump(similerity,open("similarity,pkl","wb"))

In [45]:
pickle.dump(df,open("df.pkl","wb"))